## Data Manipulation with Pandas

Pandas is a newer package built on top of NumPy, and provides an
efficient implementation of a DataFrame. DataFrames are essentially multidimensional
arrays with attached row and column labels, and often with heterogeneous types
and/or missing data. As well as offering a convenient storage interface for
labeled data, Pandas implements a number of powerful data 
operations familiar to users of both database frameworks and spreadsheet programs.
Pandas, and in particular its <b> Series and DataFrame objects </b>, 
builds on the NumPy array structure and provides efficient access to messy data 
and helps in “data munging” tasks that occupy much of a data scientist’s time.

In [3]:
import pandas as pd  # pandas : panel data , python data analysis

###### Pandas concatenate

In [4]:
#pd.concat() can be used for a simple concatenation of 
# Series or DataFrame objects,
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
print(ser1)
print(ser2)
pd.concat([ser1, ser2],axis=1)

1    A
2    B
3    C
dtype: object
4    D
5    E
6    F
dtype: object


,0,1
1,A,NaN
2,B,NaN
3,C,NaN
4,NaN,D
5,NaN,E
6,NaN,F


In [5]:
def make_df(cols, ind):
    """Quickly make a DataFrame"""
    data = {c:[str(c) + str(i) for i in ind] for c in cols}
    print(data)
    return pd.DataFrame(data, ind)

In [6]:
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1); print(df2); print(pd.concat([df1, df2],axis=1))

{'A': ['A1', 'A2'], 'B': ['B1', 'B2']}
{'A': ['A3', 'A4'], 'B': ['B3', 'B4']}
    A   B
1  A1  B1
2  A2  B2
    A   B
3  A3  B3
4  A4  B4
     A    B    A    B
1   A1   B1  NaN  NaN
2   A2   B2  NaN  NaN
3  NaN  NaN   A3   B3
4  NaN  NaN   A4   B4


In [7]:
"""By default, the concatenation takes place row-wise within the DataFrame 
(i.e.,axis=0). Like np.concatenate, pd.concat allows specification 
of an axis along which concatenation will take place."""
df3 = make_df('AB', [0, 1])
df4 = make_df('CD', [0, 1])
print(df3); print(df4); print(pd.concat([df3, df4],axis=1))

{'A': ['A0', 'A1'], 'B': ['B0', 'B1']}
{'C': ['C0', 'C1'], 'D': ['D0', 'D1']}
    A   B
0  A0  B0
1  A1  B1
    C   D
0  C0  D0
1  C1  D1
    A   B   C   D
0  A0  B0  C0  D0
1  A1  B1  C1  D1


###### Duplicate indices

In [8]:
"""One important difference between np.concatenate and pd.concat is that Pandas
concatenation preserves indices, even if the result will have duplicate indices!
Consider this simple example:"""
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # make duplicate indices!
print(x); print(y); print(pd.concat([x, y]))

{'A': ['A0', 'A1'], 'B': ['B0', 'B1']}
{'A': ['A2', 'A3'], 'B': ['B2', 'B3']}
    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


###### Caching the error

In [9]:
"""Catching the repeats as an error. If you’d like to simply verify that the indices in the
result of pd.concat() do not overlap, you can specify the verify_integrity flag.
With this set to True, the concatenation will raise an exception if there are duplicate
indices. Here is an example, where for clarity we’ll catch and print the error message:"""
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')


###### Ignoring the Index

In [10]:
"""Sometimes the index itself does not matter, and you would prefer
it to simply be ignored. You can specify this option using the ignore_index flag. With
this set to True, the concatenation will create a new integer index for the resulting
Series:"""
print(x); print(y); print(pd.concat([x, y], ignore_index=True))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


###### Adding MultiIndex keys

In [11]:
print(x); print(y); print(pd.concat([x, y], keys=['x', 'y']))

    A   B
0  A0  B0
1  A1  B1
    A   B
0  A2  B2
1  A3  B3
      A   B
x 0  A0  B0
  1  A1  B1
y 0  A2  B2
  1  A3  B3


In [12]:
"""Concatenation with joins
In the simple examples we just looked at, we were mainly concatenating DataFrames
with shared column names. In practice, data from different sources might have different
sets of column names, and pd.concat offers several options in this case. Consider
the concatenation of the following two DataFrames, which have some (but not all!)
columns in common:"""
df5 = make_df('ABC', [1, 2])
df6 = make_df('BCD', [3, 4])
print(df5); print(df6); print(pd.concat([df5, df6]))

{'A': ['A1', 'A2'], 'B': ['B1', 'B2'], 'C': ['C1', 'C2']}
{'B': ['B3', 'B4'], 'C': ['C3', 'C4'], 'D': ['D3', 'D4']}
    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
     A   B   C    D
1   A1  B1  C1  NaN
2   A2  B2  C2  NaN
3  NaN  B3  C3   D3
4  NaN  B4  C4   D4


In [13]:
print(df5); print(df6);
print(pd.concat([df5, df6], join='inner'))

    A   B   C
1  A1  B1  C1
2  A2  B2  C2
    B   C   D
3  B3  C3  D3
4  B4  C4  D4
    B   C
1  B1  C1
2  B2  C2
3  B3  C3
4  B4  C4


###### Combining Datasets: Merge and Join
One essential feature offered by Pandas is its high-performance, <b> in-memory join and
merge operations</b>. If you have ever worked with databases, you should be familiar
with this type of data interaction. The main interface for this is the pd.merge function,

In [14]:
#Categories of Join
"""The pd.merge() function implements a number of types of joins: the one-to-one,
many-to-one, and many-to-many joins. All three types of joins are accessed via an
identical call to the pd.merge() interface; the type of join performed depends on the
form of the input data. columns made as index """
#One-to-one joins
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
print(df1); print(df2)

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014


In [15]:
#To combine df1 and df2
df3 = pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [16]:
#Many-to-one joins
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
print(df3); print(df4); print(pd.merge(df3, df4))

  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve


In [17]:
#Many-to-many joins
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
print(df1); print(df5); print(pd.merge(df1, df5))                  

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization
  employee        group        skills
0      Bob   Accounting          math
1      Bob   Accounting  spreadsheets
2     Jake  Engineering        coding
3     Jake  Engineering         linux
4     Lisa  Engineering        coding
5     Lisa  Engineering         linux
6      Sue           HR  spreadsheets
7      Sue           HR  organization


In [18]:
#Specification of the Merge Key *merge inner join *concat outer join
#The on keyword default merge perfoerms imnner join
#This option works only if both the left and right DataFrames have the specified column name.
print(df1); print(df2); print(pd.merge(df1, df2, on='employee'))

  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014


In [19]:
df3 = pd.DataFrame({'name':['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
print(df2); print(df3); print(pd.merge(df2, df3, left_on='employee',right_on='name'))

  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
  employee  hire_date  name  salary
0     Lisa       2004  Lisa  120000
1      Bob       2008   Bob   70000
2     Jake       2012  Jake   80000
3      Sue       2014   Sue   90000


In [20]:
"""The result has a redundant column that we can drop if desired—for example, by
using the drop() method of DataFrames:"""
print(pd.merge(df2, df3, left_on='employee',right_on='name').drop('name',axis=1))

  employee  hire_date  salary
0     Lisa       2004  120000
1      Bob       2008   70000
2     Jake       2012   80000
3      Sue       2014   90000


In [53]:
#The left_index and right_index keywords
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
print(df1a); print(df2a)

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014


In [54]:
print(pd.merge(df1a, df2a, left_index=True, right_index=True))

                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [55]:
"""For convenience, DataFrames implement the join() method, which performs a
merge that defaults to joining on indices:"""
print(df1a); print(df2a); print(df1a.join(df2a))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
          hire_date
employee           
Lisa           2004
Bob            2008
Jake           2012
Sue            2014
                group  hire_date
employee                        
Bob        Accounting       2008
Jake      Engineering       2012
Lisa      Engineering       2004
Sue                HR       2014


In [24]:
"""If you’d like to mix indices and columns, you can combine left_index with right_on
or left_on with right_index to get the desired behavior:"""
print(df1a); print(df3);
print(pd.merge(df1a, df3, left_index=True, right_on='name'))

                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000
         group  name  salary
0   Accounting   Bob   70000
1  Engineering  Jake   80000
2  Engineering  Lisa  120000
3           HR   Sue   90000


###### Specifying Set Arithmetic for Joins

In [25]:
#by default inner join -merge
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
'food': ['fish', 'beans', 'bread']},
columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
'drink': ['wine', 'beer']},
columns=['name', 'drink'])
print(df6); print(df7); print(pd.merge(df6, df7))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
   name   food drink
0  Mary  bread  wine


In [56]:
"""We can specify this explicitly using the how keyword,
which defaults to 'inner':"""
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


In [27]:
"""Other options for the how keyword are 'outer', 'left', and 'right'. An outer join
returns a join over the union of the input columns, and fills in all missing values with
NAs:"""
print(df6); print(df7); print(pd.merge(df6, df7, how='outer'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0  Joseph    NaN  beer
1    Mary  bread  wine
2    Paul  beans   NaN
3   Peter   fish   NaN


In [28]:
print(df6); print(df7); print(pd.merge(df6, df7, how='left'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine


In [29]:
print(df6); print(df7); print(pd.merge(df6, df7, how='right'))

    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread
     name drink
0    Mary  wine
1  Joseph  beer
     name   food drink
0    Mary  bread  wine
1  Joseph    NaN  beer


###### Overlapping Column Names: The suffixes Keyword

In [30]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
'rank': [3, 1, 4, 2]})
print(df8); print(df9); print(pd.merge(df8, df9,on="rank"))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
  name_x  rank name_y
0    Bob     1   Jake
1   Jake     2    Sue
2   Lisa     3    Bob
3    Sue     4   Lisa


In [31]:
print(df8); print(df9);
print(pd.merge(df8, df9, on="name", suffixes=["_L", "_R"]))

   name  rank
0   Bob     1
1  Jake     2
2  Lisa     3
3   Sue     4
   name  rank
0   Bob     3
1  Jake     1
2  Lisa     4
3   Sue     2
   name  rank_L  rank_R
0   Bob       1       3
1  Jake       2       1
2  Lisa       3       4
3   Sue       4       2


In [32]:
#df.dropna()

In [33]:
#US States Data
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [34]:
pop.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [35]:
pop_df = pd.get_dummies(pop, columns=['ages'], dtype='int',drop_first=True)
pop_df

,state/region,year,population,ages_under18
0,AL,2012,1117489.0,1
1,AL,2012,4817528.0,0
2,AL,2010,1130966.0,1
3,AL,2010,4785570.0,0
4,AL,2011,1125763.0,1
...,...,...,...,...
2539,USA,2010,309326295.0,0
2540,USA,2011,73902222.0,1
2541,USA,2011,311582564.0,0
2542,USA,2012,73708179.0,1


In [36]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [37]:
areas.shape[0]

52

In [38]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [39]:
"""Query: Given this information, say we want to compute a relatively straightforward result:
rank US states and territories by their 2010 population density."""
merged = pd.merge(pop, abbrevs, how='outer',
left_on='state/region', right_on='abbreviation').drop('abbreviation',axis=1)
merged

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska
...,...,...,...,...,...
2539,WY,under18,1993,137458.0,Wyoming
2540,WY,total,1991,459260.0,Wyoming
2541,WY,under18,1991,136720.0,Wyoming
2542,WY,under18,1990,136078.0,Wyoming


In [40]:
merged.isnull().sum()

state/region     0
ages             0
year             0
population      20
state           96
dtype: int64

In [41]:
"""It appears that all the null population values are from Puerto Rico prior to the year
2000; this is likely due to this data not being available from the original source."""

"""The statement filters the merged DataFrame for rows where the ‘population’ 
column contains missing values (NaN) and returns the first 5 rows of this filtered DataFrame.."""

merged[merged['population'].isnull()].head()


,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


In [42]:
"""we see also that some of the new state entries are also null, which
means that there was no corresponding entry in the abbrevs key! Let’s figure out
which regions lack this match"""

"""The statement filters the merged DataFrame for rows where the ‘state’ column 
has missing values, and then retrieves the unique values from the ‘state/region’column in those filtered rows. 
This will give a list of unique state/region codes or identifiers that correspond to missing state information."""

merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [43]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [44]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [45]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [46]:
merged.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


In [47]:
pd.set_option('display.max_rows', 3000)

In [48]:
"""Now we can merge the result with the area data using a similar procedure. Examining
our results, we will want to join on the state column in both:"""
final = pd.merge(merged, areas, on='state', how='left')
final

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0
5,AK,total,1994,603308.0,Alaska,656425.0
6,AK,under18,1994,187439.0,Alaska,656425.0
7,AK,total,1991,570193.0,Alaska,656425.0
8,AK,total,1993,599434.0,Alaska,656425.0
9,AK,under18,1993,187190.0,Alaska,656425.0


In [49]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [50]:
"""This statement filters the final DataFrame for rows where the ‘area (sq. mi)’ column has missing values and returns the unique values from the ‘state’ column in those rows. 
This will give a list of unique states that have missing data in the ‘area (sq. mi)’ field."""
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [51]:
"""We see that our areas DataFrame does not contain the area of the United States as a
whole. We could insert the appropriate value (using the sum of all state areas, for
instance), but in this case we’ll just drop the null values because the population density
of the entire United States is not relevant to our current discussion:"""
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [52]:
# Calculate correlation matrix
correlation_matrix = final[['population', 'area (sq. mi)']].corr()

# Generate a heatmap for the correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix,annot=True, linewidths=0.5,cmap='viridis')
plt.title('Heatmap of Population, Area Correlation')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
!pip install numexpr

In [ ]:
#data2010 = final[(final.year == 2010) & (final.ages == 'total')]
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

In [ ]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

In [ ]:
#end list
density.tail()

###### Aggregation and Grouping

In [63]:
#Planets Data
"""It gives information on planets that astronomers
have discovered around other stars (known as extrasolar planets or exoplanets for
short). It can be downloaded with a simple Seaborn command:"""
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [64]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [65]:
planets[['method','year']]

,method,year
0,Radial Velocity,2006
1,Radial Velocity,2008
2,Radial Velocity,2011
3,Radial Velocity,2007
4,Radial Velocity,2009
5,Radial Velocity,2008
6,Radial Velocity,2002
7,Radial Velocity,1996
8,Radial Velocity,2008
9,Radial Velocity,2010


In [66]:
planets.columns

Index(['method', 'number', 'orbital_period', 'mass', 'distance', 'year'], dtype='object')

In [67]:
planets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [68]:
"""there is a convenience method describe() that computes several common aggregates for
each column and returns the result."""
planets.describe(include='all')

,method,number,orbital_period,mass,distance,year
count,1035,1035.000000,992.000000,513.000000,808.000000,1035.000000
unique,10,NaN,NaN,NaN,NaN,NaN
top,Radial Velocity,NaN,NaN,NaN,NaN,NaN
freq,553,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,NaN,1.240976,26014.728304,3.818617,733.116493,3.972567
min,NaN,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,NaN,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,NaN,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,NaN,2.000000,526.005000,3.040000,178.500000,2012.000000


In [69]:
planets.isnull().any()

method            False
number            False
orbital_period     True
mass               True
distance           True
year              False
dtype: bool

In [70]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


<pre><b>Listing of Pandas aggregation methods</b>
Aggregation           Description
count()               Total number of items
first(), last()       First and last item
mean(), median()      Mean and median
min(), max()          Minimum and maximum
std(), var()          Standard deviation and variance
mad()                 Mean absolute deviation
prod()                Product of all items
sum()                 Sum of all items</pre>

###### Group by: Split, Apply and Combine
<img src="Group by.png">

In [58]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
'data': range(1,7)}, columns=['key', 'data'])

In [59]:
df

,key,data
0,A,1
1,B,2
2,C,3
3,A,4
4,B,5
5,C,6


In [60]:
df.groupby('key')

In [61]:
df.groupby('key').sum()

,data
key,
A,5
B,7
C,9


In [72]:
planets.groupby('method')

In [73]:
#column indexing
planets.groupby('method')['orbital_period']

In [74]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [75]:
df = pd.read_json('2024.json')
df.shape

(1000, 9)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1000 non-null   object 
 1   code            1000 non-null   object 
 2   logo            1000 non-null   object 
 3   rank            1000 non-null   object 
 4   region          1000 non-null   object 
 5   region_code     1000 non-null   object 
 6   region_ranking  1000 non-null   object 
 7   score           100 non-null    float64
 8   breakdown       1000 non-null   object 
dtypes: float64(1), object(8)
memory usage: 70.4+ KB


In [77]:
df.head(10)

,name,code,logo,rank,region,region_code,region_ranking,score,breakdown
0,Harvard University,harvard-university,logo/032bd1b77.png,1,United States,us,1,100.0,"{'Alumni': 100, 'Award': 97.1, 'HiCi': 100, 'N..."
1,Stanford University,stanford-university,logo/13de8913b.png,2,United States,us,2,75.6,"{'Alumni': 45.3, 'Award': 88.1, 'HiCi': 73.1, ..."
2,Massachusetts Institute of Technology (MIT),massachusetts-institute-of-technology-mit,logo/79165fd8b.png,3,United States,us,3,69.5,"{'Alumni': 71.9, 'Award': 82.8, 'HiCi': 55.2, ..."
3,University of Cambridge,university-of-cambridge,logo/8d9861b69.png,4,United Kingdom,gb,1,67.8,"{'Alumni': 78.2, 'Award': 90.8, 'HiCi': 45.1, ..."
4,"University of California, Berkeley",university-of-california-berkeley,logo/0ff179fb8.png,5,United States,us,4,62.0,"{'Alumni': 65.3, 'Award': 76.5, 'HiCi': 46.9, ..."
5,University of Oxford,university-of-oxford,logo/c1d967477.png,6,United Kingdom,gb,2,60.0,"{'Alumni': 49.5, 'Award': 59.7, 'HiCi': 51.3, ..."
6,Princeton University,princeton-university,logo/b1dad5288.png,7,United States,us,5,58.8,"{'Alumni': 61.2, 'Award': 100, 'HiCi': 26, 'N&..."
7,California Institute of Technology,california-institute-of-technology,logo/c956f3d98.png,8,United States,us,6-7,55.1,"{'Alumni': 55.7, 'Award': 63.4, 'HiCi': 29.8, ..."
8,Columbia University,columbia-university,logo/ce3421720.png,8,United States,us,6-7,55.1,"{'Alumni': 59.4, 'Award': 63.2, 'HiCi': 42.7, ..."
9,University of Chicago,university-of-chicago,logo/b4f6eec47.png,10,United States,us,8,54.4,"{'Alumni': 59, 'Award': 84.5, 'HiCi': 32.5, 'N..."


In [78]:
import pandas as pd
import sqlite3

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('diabetes_data_all_patient.csv')

# Create a connection to SQLite database
conn = sqlite3.connect('my_database.db')

# Write the data to a table named 'diabetes'
df.to_sql('diabetes', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [79]:
# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')

# Create a cursor object
cursor = conn.cursor()

# Execute a query to get the table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch and print the list of tables
tables = cursor.fetchall()
print(tables)

# Close the connection
conn.close()

[('diabetes',)]


In [80]:

# Connect to the SQLite database
conn = sqlite3.connect('my_database.db')

# Write a SQL query to fetch the data
query = "SELECT * FROM diabetes"

# Read the data into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df)

       Unnamed: 0  patient_id        date   time  code value
0               0           1  04-21-1991   9:09    58   100
1               1           1  04-21-1991   9:09    33     9
2               2           1  04-21-1991   9:09    34    13
3               3           1  04-21-1991  17:08    62   119
4               4           1  04-21-1991  17:08    33     7
...           ...         ...         ...    ...   ...   ...
29325         336          70  05-09-1989  08:00    33   1.0
29326         337          70  05-09-1989  08:00    34   7.0
29327         338          70  05-10-1989  08:00    34   7.0
29328         339          70  05-11-1989  08:00    34   7.0
29329         340          70  05-12-1989  08:00    34   7.0

[29330 rows x 6 columns]


In [81]:
#%pip install jupytext
#!jupytext --to notebook seaborn_visualization_examples.py

In [ ]:
#%pip install jupytext
#!jupytext --to notebook plotly_visualization_examples.py